In [1]:
import os
import glob
import pandas as pd
import yfinance as yf
import numpy as np

c:\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [ ]:
#%pip install yfinance --upgrade --no-cache-dir --user

In [41]:
InfosToCopy = pd.read_csv('orders.csv', delimiter=";", header=0)
assetsList = sorted(InfosToCopy['Ativo'].unique())

for position in range(len(assetsList)):
    data = yf.download(assetsList[position]+".SA", start="2022-09-26")
    local = "output1/"+assetsList[position]+".csv"
    data.to_csv(local,index=True,header=True)
    #print(position, len(assetsList))    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
#quantityHistory


In [ ]:
InfosToCopy = pd.read_csv('lista2.csv', delimiter=";", header=None)
for position in range(len(InfosToCopy)):
    data = yf.download(InfosToCopy.iloc[position][0]+".SA", period="max")
    local = "output1/"+InfosToCopy.iloc[position][0]+".csv"
    data.to_csv(local,index=True,header=True)
    print(position, len(InfosToCopy))

In [ ]:
def build_RSI(data, column='Close', window=9, path="XXXX"):    

    # Establish gains and losses for each day
    data['Variation'] = data[column].diff()
    data = data[1:]
    data['Gain'] = np.where(data['Variation'] > 0, data['Variation'], 0)
    data['Loss'] = np.where(data['Variation'] < 0, data['Variation'], 0)

    # Calculate simple averages so we can initialize the classic averages
    simple_avg_gain = data['Gain'].rolling(window).mean()
    simple_avg_loss = data['Loss'].abs().rolling(window).mean()

    classic_avg_gain = simple_avg_gain.copy()
    classic_avg_loss = simple_avg_loss.copy()

    for i in range(window, len(classic_avg_gain)):
        classic_avg_gain[i] = (classic_avg_gain[i - 1] * (window - 1) + data['Gain'].iloc[i]) / window
        classic_avg_loss[i] = (classic_avg_loss[i - 1] * (window - 1) + data['Loss'].abs().iloc[i]) / window
    
    # Calculate the RSI
    data['Simple RS'] = simple_avg_gain / simple_avg_loss
    data['Classic RS'] = classic_avg_gain / classic_avg_loss

    data['Simple RSIf'] = 100 - (100 / (1 + data['Simple RS']))
    data['Classic RSIf'] = 100 - (100 / (1 + data['Classic RS']))

    data['Ticker'] = path.split("\\")[-1].split(".")[0]

    low_min = data['Low'].rolling(14).min()
    high_max = data['High'].rolling(14).max()
    
    # Fast Stochastic
    data['k_fast'] = 100 * (data['Close'] - low_min)/(high_max - low_min)

    # Slow Stochastic
    data['d_slow'] = data['k_fast'].rolling(3).mean()

    # Export to CSV
    local = "output2/"+path.split("\\")[-1]
    data.to_csv(local, index=True, sep=",", encoding='utf-8')

In [ ]:
path = "output1"
csv_files = glob.glob(os.path.join(path, "*.csv"))

for files in csv_files:
    df = pd.read_csv(files, header='infer', index_col=0)
    build_RSI(data=df, column="Close", window=9, path=files.split("\\")[-1])

In [ ]:
df = pd.read_csv("output2\BRFS3.csv")
#rslt_df = df[(df['Classic RSIf'] <= 15)] #164
#rslt_df = df[(df['Classic RSIf'] <= 15) & (df['k_slow'] <= 15)] #142
rslt_df = df[(df['Classic RSIf'] <= 15) & (df['d_slow'] <= 15)] #145
df.tail(30)

In [ ]:
df = pd.read_csv("output2\ITUB4.csv")
rslt_df = df[(df['Classic RSIf'] <= 15)] #187
#rslt_df = df[(df['Classic RSIf'] <= 15) & (df['d_slow'] <= 15)] #145
rslt_df.tail(30)

In [24]:
path = "output2"
csv_files = glob.glob(os.path.join(path, "*.csv"))

for files in csv_files:
    df = pd.read_csv(files, header='infer', index_col=0)
    rslt_df = df[(df['Classic RSIf'] <= 15) & (df['d_slow'] <= 15)]
    rslt_df.to_csv('output3/filter1c.csv', mode='a', header=True)
    rslt_df = df[(df['Classic RSIf'] >= 85) & (df['d_slow'] >= 85)]
    rslt_df.to_csv('output3/filter1v.csv', mode='a', header=True)
    #print(files)
    #build_RSI(data=df, column="Close", window=9, path=files.split("\\")[-1])

C:\Users\hfeli\AppData\Roaming\Python\Python310\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
C:\Users\hfeli\AppData\Roaming\Python\Python310\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
C:\Users\hfeli\AppData\Roaming\Python\Python310\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
C:\Users\hfeli\AppData\Roaming\Python\Python310\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
C:\Users\hfeli\AppData\Roaming\Python\Python310\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
C:\Users\hfeli\AppData\Roaming\Python\Python310\site-packages\pandas\core\internals\blocks.py:2323: 

In [35]:
df = pd.read_csv("output3/filter1c.csv")
df = df[(df['Date'] != "Date")]
df['Open'] = pd.to_numeric(df['Open'])
df['High'] = pd.to_numeric(df['High'])
df['Low'] = pd.to_numeric(df['Low'])
df['Close'] = pd.to_numeric(df['Close'])
df['Adj Close'] = pd.to_numeric(df['Adj Close'])
df['Volume'] = pd.to_numeric(df['Volume'])
df['Variation'] = pd.to_numeric(df['Variation'])
df['Gain'] = pd.to_numeric(df['Gain'])
df['Loss'] = pd.to_numeric(df['Loss'])
df['Simple RS'] = pd.to_numeric(df['Simple RS'])
df['Classic RS'] = pd.to_numeric(df['Classic RS'])
df['Simple RSIf'] = pd.to_numeric(df['Simple RSIf'])
df['Classic RSIf'] = pd.to_numeric(df['Classic RSIf'])
df['k_fast'] = pd.to_numeric(df['k_fast'])
df['d_slow'] = pd.to_numeric(df['d_slow'])
df['Category'] = "C"
df = df[(df['Volume'] >= 1000000)]
df.to_csv('output3/filter2c.csv', header=True)

# Volume >= 1.000.000 - 1716 linhas
# Qualquer Volume - 58 mil linhas
# Volume >= 10.000 - 6006 linhas

C:\Users\hfeli\AppData\Roaming\Python\Python310\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [36]:
df = pd.read_csv("output3/filter1v.csv")
df = df[(df['Date'] != "Date")]
df['Open'] = pd.to_numeric(df['Open'])
df['High'] = pd.to_numeric(df['High'])
df['Low'] = pd.to_numeric(df['Low'])
df['Close'] = pd.to_numeric(df['Close'])
df['Adj Close'] = pd.to_numeric(df['Adj Close'])
df['Volume'] = pd.to_numeric(df['Volume'])
df['Variation'] = pd.to_numeric(df['Variation'])
df['Gain'] = pd.to_numeric(df['Gain'])
df['Loss'] = pd.to_numeric(df['Loss'])
df['Simple RS'] = pd.to_numeric(df['Simple RS'])
df['Classic RS'] = pd.to_numeric(df['Classic RS'])
df['Simple RSIf'] = pd.to_numeric(df['Simple RSIf'])
df['Classic RSIf'] = pd.to_numeric(df['Classic RSIf'])
df['k_fast'] = pd.to_numeric(df['k_fast'])
df['d_slow'] = pd.to_numeric(df['d_slow'])
df['Category'] = "V"
df = df[(df['Volume'] >= 1000000)]
df.to_csv('output3/filter2v.csv', header=True)

# Volume >= 1.000.000 - 1716 linhas
# Qualquer Volume - 58 mil linhas
# Volume >= 10.000 - 6006 linhas

C:\Users\hfeli\AppData\Roaming\Python\Python310\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [30]:
df = pd.read_csv("output3/filter2v.csv", index_col=0)
df.head(30)

,Date,Open,High,Low,Close,Adj Close,Volume,Variation,Gain,Loss,Simple RS,Classic RS,Simple RSIf,Classic RSIf,Ticker,k_fast,d_slow
9,2020-01-10,20.000000,20.990000,19.980000,20.950001,20.777952,1498800,1.050001,1.050001,0.000000,50.167117,6.816475,98.045620,87.206510,AALR3,98.738202,92.697518
10,2020-01-13,20.959999,21.930000,20.959999,21.879999,21.700314,1170300,0.929998,0.929998,0.000000,65.000540,9.547961,98.484861,90.519495,AALR3,98.783427,97.644819
20,2020-06-08,11.990000,13.250000,11.960000,13.250000,13.250000,1240900,1.450000,1.450000,0.000000,14.483895,6.660633,93.541677,86.946248,AALR3,100.000000,94.398391
21,2020-06-09,12.760000,13.690000,12.240000,13.320000,13.320000,1235300,0.070000,0.070000,0.000000,19.000016,6.802410,95.000004,87.183447,AALR3,92.911879,93.285683
22,2021-05-20,10.890000,11.900000,10.830000,11.510000,11.510000,3359100,0.650001,0.650001,0.000000,9.062507,6.629214,90.062119,86.892489,AALR3,87.579635,94.429593
23,2021-08-23,14.530000,14.840000,13.610000,14.690000,14.690000,1925100,0.190000,0.190000,0.000000,5.470586,5.802713,84.545449,85.299984,AALR3,97.311818,91.515864
24,2021-08-25,14.490000,15.450000,14.010000,15.310000,15.310000,1850800,0.710000,0.710000,0.000000,6.850576,6.187314,87.262081,86.086597,AALR3,97.738297,95.084273
32,2009-01-06,4.123022,4.564775,4.049397,4.491149,2.309778,1811331,0.449115,0.449115,0.000000,inf,16.890712,100.000000,94.410508,ABCB4,96.153814,96.884605
33,2009-01-07,4.417524,4.491149,4.329173,4.454336,2.290845,1075579,-0.036813,0.000000,-0.036813,34.999909,10.745827,97.222215,91.486338,ABCB4,93.877513,94.843776
40,2009-04-03,4.196648,4.410161,4.159835,4.410161,2.268127,1291266,0.139888,0.139888,0.000000,28.799934,8.616281,96.644288,89.600969,ABCB4,100.000000,97.590378


In [48]:
dfv = pd.read_csv("output3/filter2v.csv", index_col=0)
dfc = pd.read_csv("output3/filter2c.csv", index_col=0)

dfc.to_csv('output3/filter3.csv', mode='a', header=True)
dfv.to_csv('output3/filter3.csv', mode='a', header=False)

df3 = pd.read_csv("output3/filter3.csv", index_col=0)
df3.tail(30)

C:\Users\hfeli\AppData\Roaming\Python\Python310\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


,Date,Open,High,Low,Close,Adj Close,Volume,Variation,Gain,Loss,Simple RS,Classic RS,Simple RSIf,Classic RSIf,Ticker,k_fast,d_slow,Category
66523,2012-08-09,8.883333,9.583333,8.883333,9.410000,6.978785,4479000,0.843334,0.843334,0.000000,inf,9.653793,100.000000,90.613671,YDUQ3,89.344275,96.448092,V
66524,2012-08-10,9.343333,9.766666,9.306666,9.766666,7.243301,4928400,0.356667,0.356667,0.000000,inf,13.144946,100.000000,92.930337,YDUQ3,100.000000,96.448092,V
66526,2012-08-14,9.983333,9.966666,9.860000,9.966666,7.391627,1315200,0.200000,0.200000,0.000000,inf,15.622602,100.000000,93.984095,YDUQ3,100.000000,100.000000,V
66527,2012-08-15,9.883333,9.976666,9.866666,9.900000,7.342187,1339800,-0.066667,0.000000,-0.066667,21.000029,8.098300,95.454551,89.008936,YDUQ3,95.764264,98.588088,V
66528,2012-08-16,9.986666,10.243333,9.933333,10.146666,7.525122,1162200,0.246666,0.246666,0.000000,24.700021,10.103078,96.108953,90.993488,YDUQ3,95.158567,96.974277,V
66529,2012-10-02,11.406666,11.746666,11.406666,11.666666,8.652408,1090200,0.296666,0.296666,0.000000,16.714125,7.754739,94.354788,88.577614,YDUQ3,91.946313,93.533922,V
66532,2012-10-05,11.886666,12.033333,11.780000,12.033333,8.924340,1542000,0.166667,0.166667,0.000000,31.999571,12.615457,96.969658,92.655406,YDUQ3,100.000000,95.173154,V
66533,2013-02-15,15.460000,16.113333,15.460000,15.900000,11.791996,3763200,0.440000,0.440000,0.000000,5.615380,5.882265,84.883710,85.469900,YDUQ3,89.743599,90.602052,V
66534,2013-02-18,15.906666,16.063334,15.466666,15.993333,11.861216,3381900,0.093333,0.093333,0.000000,5.089737,6.308315,83.578930,86.316955,YDUQ3,93.835622,90.893529,V
66535,2013-02-27,16.219999,16.406666,16.143333,16.406666,12.167758,1664400,0.190001,0.190001,0.000000,6.461538,6.051821,86.597938,85.819265,YDUQ3,100.000000,92.173198,V


In [49]:
header = ['Ticker', 'Date', 'Category', 'Adj Close']
df3.to_csv('output3/filter4.csv', header=True, columns=header)

In [52]:
df4 = pd.read_csv("output3/filter4.csv", index_col=0)
df4.head(30)

,Ticker,Date,Category,Adj Close
16,ABCB4,2020-03-09,C,13.706721
17,ABCB4,2020-03-12,C,11.891090
48,ABEV3,2008-07-04,C,2.039221
49,ABEV3,2008-07-07,C,2.034976
50,ABEV3,2016-11-01,C,14.711755
51,ABEV3,2016-11-03,C,14.366632
52,ABEV3,2016-11-04,C,14.222162
53,ABEV3,2018-05-18,C,17.311928
54,ABEV3,2018-05-21,C,17.035002
55,ABEV3,2018-05-22,C,16.909130


In [3]:
InfosToCopy2 = pd.read_csv('listaComRSI.csv', delimiter=";", header=None)
FlagOnOff = 0
for position in range(len(InfosToCopy2)):
    local = "output1/"+InfosToCopy2.iloc[position][0]+".csv"
    df5 = pd.read_csv(local, index_col=0)
    df5['Ticker'] = InfosToCopy2.iloc[position][0]
    if FlagOnOff == 0:
        df5.to_csv('output3/filter5.csv', mode='a', header=True)
        FlagOnOff = 1
    else:
        df5.to_csv('output3/filter5.csv', mode='a', header=False)
    